In [2]:
import pandas as pd
path=('/home/kakashi/Documents/capstone/postal.csv')
df=pd.read_csv(path)
df.drop(columns='Unnamed: 0',inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [3]:
dl=pd.read_csv('/home/kakashi/Downloads/Geospatial_Coordinates.csv')
dl.rename(columns={'Postal Code':'PostalCode'},inplace=True)
dl.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [4]:
ds=pd.merge(df,dl, on=['PostalCode'])
ds.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
!pip install geopy
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [6]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [7]:
import folium
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighborhood in zip(ds['Latitude'], ds['Longitude'], ds['Borough'], ds['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork



Lets split the data for simplicity. Only selecting North York as Borough

In [8]:
NY = ds[ds['Borough'] == 'North York'].reset_index(drop=True)
NY.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [9]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [10]:
# create map of Manhattan using latitude and longitude values
map_NY = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(NY['Latitude'], NY['Longitude'], NY['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NY)  
    
map_NY

In [11]:
CLIENT_ID='JTZNBF4GA1SS3C1EY4P1XSHF51ZYKFWNOIG442D2PXDZARPO'
CLIENT_SECRET='FV1YA5WCZA2FMOH5PSWHHVJ1A1HO2HDQK4RS51ZPQYJLYHOS'
VERSION='20180605'
LIMIT=20

In [12]:
NY.loc[0,'Neighborhood']

'Parkwoods'

In [13]:
NY_Lat=NY.loc[0,'Latitude']
NY_Lon=NY.loc[0,'Longitude']
NY_Name=NY.loc[0,'Neighborhood']
print('Latidue and longitude of {} are {}, {}'.format(NY_Name,NY_Lat,NY_Lon))

Latidue and longitude of Parkwoods are 43.7532586, -79.3296565


Now lets get top 20 venues that are in Parkwoods within a radius of 500 meters

In [14]:
LIMIT=20
radius=500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    NY_Lat, 
    NY_Lon, 
    radius, 
    LIMIT)

In [16]:
results=requests.get(url).json()

In [17]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/kakashi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [19]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


Explore Neighborhoods in North York

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
NY_venues = getNearbyVenues(names=NY['Neighborhood'],
                                   latitudes=NY['Latitude'],
                                   longitudes=NY['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [24]:
NY_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",20,20,20,20,20,20
Don Mills,25,25,25,25,25,25
Downsview,17,17,17,17,17,17
"Fairview, Henry Farm, Oriole",20,20,20,20,20,20
Glencairn,5,5,5,5,5,5
Hillcrest Village,5,5,5,5,5,5
Humber Summit,2,2,2,2,2,2


In [26]:
print('There are {} uniques categories.'.format(len(NY_venues['Venue Category'].unique())))

There are 91 uniques categories.


Analyse each Neighborhood

In [28]:
NY_A = pd.get_dummies(NY_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to datafram
NY_A['Neighborhood'] = NY_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [NY_A.columns[-1]] + list(NY_A.columns[:-1])
NY_A = NY_A[fixed_columns]

NY_A.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Bridal Shop,Burger Joint,Business Service,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Martial Arts School,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
NY_grouped = NY_A.groupby('Neighborhood').mean().reset_index()
NY_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Beer Store,Bike Shop,Boutique,Bridal Shop,Burger Joint,Business Service,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Event Space,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Martial Arts School,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Speakeasy,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tennis Court,Thai Restaurant,Theater,Toy / Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.100000,0.000000,0.000000,0.00,0.00,0.000000,0.05,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.100000,0.00,0.000000,0.0,0.05,0.00,0.00,0.05,0.00,0.0,0.00,0.000000,0.0,0.00,0.000000,0.05,0.05,0.000000,0.05,0.000000,0.0,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.05,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.05,0.000000,0.05,0.00,0.000000,0.00,0.05,0.050000,0.00,0.00,0.000000,0.00,0.00,0.05,0.00,0.05,0.050000,0.000000,0.00,0.00,0.05,0.05,0.00,0.00,0.00,0.00,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.25,0.000000,0.25,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.000000,0.0,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.25,0.00,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.05,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.05,0.05,0.000000,0.00,0.00,0.000000,0.100000,0.05,0.000000,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.000000,0.0,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.0,0.05,0.050000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.05,0.000000,0.10,0.00,0.05,0.050000,0.00,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.05,0.000000,0.00,0.00,0.000000,0.05,0.00,0.05,0.00,0.10,0.000000,0.000000,0.00,0.00,0.00,0.05,0.00,0.05,0.00,0.00,0.000000,0.000000
3,Don Mills,0.000000,0.000000,0.00,0.04,0.00,0.04,0.000000,0.000000,0.000000,0.000000,0.000000,0.08,0.04,0.000000,0.00,0.00,0.000000,0.00,0.04,0.040000,0.04,0.00,0.040000,0.080000,0.00,0.000000,0.0,0.00,0.00,0.04,0.00,0.04,0.0,0.00,0.000000,0.0,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.12,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.04,0.08,0.00,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.04,0.000000,0.00,0.00,0.04,0.00,0.04,0.000000,0.000000,0.04,0.00,0.04,0.00,0.04,0.00,0.00,0.00,0.000000,0.000000
4,Downsview,0.000000,0.058824,0.00,0.00,0.00,0.00,0.058824,0.000000,0.058824,0.000000,0.058824,0.00,0.00,0.000000,0.00,0.00,0.058824,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0

In [30]:
num_top_venues = 5

for hood in NY_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = NY_grouped[NY_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
         venue  freq
0         Bank  0.10
1  Coffee Shop  0.10
2  Pizza Place  0.05
3  Bridal Shop  0.05
4  Gas Station  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.10
1  Italian Restaurant  0.10
2         Coffee Shop  0.10
3       Grocery Store  0.05
4   Indian Restaurant  0.05


----Don Mills----
                 venue  freq
0                  Gym  0.12
1          Coffee Shop  0.08
2  Japanese Restaurant  0.08
3           Beer Store  0.08
4       Sandwich Place  0.04


----Downsview----
            venue  freq
0   Grocery Store  0.18
1            Park  0.12
2       Speakeasy  0.12
3  Baseball Field  0.06
4           Hotel  0.06


----Fairview, Henry Farm, Oriole----
         

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = NY_grouped['Neighborhood']

for ind in np.arange(NY_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NY_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Mobile Phone Shop,Gas Station,Diner,Ice Cream Shop,Deli / Bodega,Middle Eastern Restaurant,Frozen Yogurt Shop,Bridal Shop
1,Bayview Village,Japanese Restaurant,Chinese Restaurant,Café,Bank,Electronics Store,Convenience Store,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Grocery Store,Restaurant,Greek Restaurant,Indian Restaurant,Juice Bar,Liquor Store,Comfort Food Restaurant
3,Don Mills,Gym,Coffee Shop,Beer Store,Japanese Restaurant,Sandwich Place,Discount Store,Bike Shop,Pool,Italian Restaurant,Restaurant
4,Downsview,Grocery Store,Park,Speakeasy,Bank,Liquor Store,Business Service,Hotel,Baseball Field,Shopping Mall,Home Service


Cluster Neighborhoods

In [50]:
kclusters = 5

NY_grouped_clustering = NY_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NY_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 4, 4, 4, 4, 4, 3, 2], dtype=int32)

In [66]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels6', kmeans.labels_)

NY_merged = NY

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
NY_merged = NY_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

NY_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels6,Cluster Labels5,Cluster Labels4,Cluster Labels3,Cluster Labels2,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Park,Food & Drink Shop,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store,Dim Sum Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,4.0,4.0,4.0,4.0,4.0,4.0,4.0,Coffee Shop,Pizza Place,Hockey Arena,French Restaurant,Portuguese Restaurant,Intersection,Diner,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,4.0,4.0,4.0,4.0,4.0,4.0,4.0,Clothing Store,Women's Store,Vietnamese Restaurant,Boutique,Coffee Shop,Event Space,Furniture / Home Store,Gift Shop,Accessories Store,Sporting Goods Shop
3,M3B,North York,Don Mills,43.745906,-79.352188,4.0,4.0,4.0,4.0,4.0,4.0,4.0,Gym,Coffee Shop,Beer Store,Japanese Restaurant,Sandwich Place,Discount Store,Bike Shop,Pool,Italian Restaurant,Restaurant
4,M6B,North York,Glencairn,43.709577,-79.445073,4.0,4.0,4.0,4.0,4.0,4.0,4.0,Park,Japanese Restaurant,Pizza Place,Bakery,Pub,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Deli / Bodega


In [71]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(NY_merged['Latitude'], NY_merged['Longitude'], NY_merged['Neighborhood'], NY_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        color=rainbow[-1],
        popup=label,
        fill_color=rainbow[-3],
        fill=True,
        
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [74]:
NY_merged.loc[NY_merged['Cluster Labels'] == 0, NY_merged.columns[[1] + list(range(5, NY.shape[1]))]]

,Borough
0,North York
14,North York
22,North York


In [76]:
NY_merged.loc[NY_merged['Cluster Labels'] == 1, NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Borough,Cluster Labels6,Cluster Labels5,Cluster Labels4,Cluster Labels3,Cluster Labels2,Cluster Labels1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,North York,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Martial Arts School,Women's Store,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner
